In [10]:
import re
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()